# OOD Evaluation of Toxic Comment Classifiers
## Final Project Experiments & Results

This notebook reproduces all experiments, results, and plots for the project.
It covers:
1.  **Setup**: Environment and Data.
2.  **Baselines**: TF-IDF + Logistic Regression / SVM.
3.  **Models**: RoBERTa (In-domain and Cross-domain).
4.  **Fairness**: Demographic Parity and Equal Opportunity analysis.
5.  **Analysis**: Final plots and tables.

**Datasets**: Civil Comments, HateXplain, Jigsaw.
**Goal**: Evaluate OOD generalization and fairness.

In [ ]:
# @title 1. Setup & Installation
import os
import sys
import subprocess
from pathlib import Path

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("Running in Local Environment")

# Install dependencies
if IN_COLAB:
    print("Installing dependencies...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
    # Install specific versions if needed, e.g. transformers
    subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers", "scikit-learn", "pandas", "matplotlib", "seaborn", "torch"])

# Mount Drive if needed (optional)
# from google.colab import drive
# drive.mount('/content/drive')

# Set paths
REPO_DIR = Path(os.getcwd())
if IN_COLAB:
    # Assuming repo is cloned to /content/ood-eval-toxic-classifiers or similar
    # If not, clone it:
    if not (REPO_DIR / "scripts").exists():
        print("Cloning repository...")
        !git clone https://github.com/aayushakumar/ood-eval-toxic-classifiers.git
        os.chdir("ood-eval-toxic-classifiers")
        REPO_DIR = Path(os.getcwd())

DATA_DIR = REPO_DIR / "data"
EXPERIMENTS_DIR = REPO_DIR / "experiments"
SCRIPTS_DIR = REPO_DIR / "scripts"

EXPERIMENTS_DIR.mkdir(exist_ok=True)

print(f"Working Directory: {REPO_DIR}")
print(f"Data Directory: {DATA_DIR}")
print(f"Experiments Directory: {EXPERIMENTS_DIR}")

In [ ]:
# @title 2. Data Verification
import pandas as pd

REQUIRED_FILES = [
    "civil_train.csv", "civil_val.csv", "civil_test.csv",
    "hatexplain_train.csv", "hatexplain_val.csv", "hatexplain_test.csv",
    "jigsaw_train.csv", "jigsaw_val.csv", "jigsaw_test.csv"
]

FULL_FILES = [
    "jigsaw_test_full.csv",  # Required for fairness analysis
    "jigsaw_train_full.csv",
    "jigsaw_val_full.csv",
]

missing_files = []
for fname in REQUIRED_FILES:
    if not (DATA_DIR / fname).exists():
        missing_files.append(fname)

if missing_files:
    print(f"❌ WARNING: Missing data files: {missing_files}")
    print("Please ensure data is uploaded to the 'data/' directory.")
else:
    print("✓ All required data files found.")

# Check full files for fairness
missing_full = []
for fname in FULL_FILES:
    if not (DATA_DIR / fname).exists():
        missing_full.append(fname)

if missing_full:
    print(f"\n⚠️  Missing full data files (needed for fairness): {missing_full}")
else:
    print("✓ Full data files for fairness analysis found.")

# Dataset statistics
print("\n" + "=" * 60)
print("DATASET STATISTICS")
print("=" * 60)

for dataset in ["civil", "hatexplain", "jigsaw"]:
    print(f"\n{dataset.upper()}:")
    for split in ["train", "val", "test"]:
        fpath = DATA_DIR / f"{dataset}_{split}.csv"
        if fpath.exists():
            df = pd.read_csv(fpath)
            n_toxic = (df["label"] == 1).sum()
            n_total = len(df)
            print(f"  {split:5s}: {n_total:6d} samples, {n_toxic:5d} toxic ({100*n_toxic/n_total:.1f}%)")

## 2.5 Exploratory Data Analysis (EDA)

Visualize dataset characteristics: label distributions, text lengths, and class imbalance.

In [ ]:
# @title EDA: Dataset Visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

datasets_info = []

for idx, dataset in enumerate(["civil", "hatexplain", "jigsaw"]):
    # Load train data
    train_path = DATA_DIR / f"{dataset}_train.csv"
    if not train_path.exists():
        continue
    df = pd.read_csv(train_path)
    
    # Text lengths
    df["text_len"] = df["text"].astype(str).apply(len)
    df["word_count"] = df["text"].astype(str).apply(lambda x: len(x.split()))
    
    # Store stats
    datasets_info.append({
        "Dataset": dataset,
        "Total": len(df),
        "Toxic": (df["label"] == 1).sum(),
        "Non-Toxic": (df["label"] == 0).sum(),
        "Toxic %": 100 * (df["label"] == 1).mean(),
        "Avg Words": df["word_count"].mean(),
        "Avg Chars": df["text_len"].mean()
    })
    
    # Plot 1: Label distribution
    ax1 = axes[0, idx]
    labels, counts = np.unique(df["label"], return_counts=True)
    colors = ["#2ecc71", "#e74c3c"]
    ax1.bar(["Non-Toxic", "Toxic"], counts, color=colors)
    ax1.set_title(f"{dataset.upper()}: Label Distribution")
    ax1.set_ylabel("Count")
    for i, (l, c) in enumerate(zip(labels, counts)):
        ax1.text(i, c + 100, f"{c:,}", ha="center", fontsize=10)
    
    # Plot 2: Text length distribution
    ax2 = axes[1, idx]
    ax2.hist(df["word_count"], bins=50, alpha=0.7, color="#3498db", edgecolor="black")
    ax2.axvline(df["word_count"].mean(), color="red", linestyle="--", label=f"Mean: {df['word_count'].mean():.1f}")
    ax2.set_title(f"{dataset.upper()}: Word Count Distribution")
    ax2.set_xlabel("Word Count")
    ax2.set_ylabel("Frequency")
    ax2.legend()
    ax2.set_xlim(0, 200)

plt.tight_layout()
plt.savefig(EXPERIMENTS_DIR / "eda_plots.png", dpi=150, bbox_inches="tight")
plt.show()

# Print summary table
print("\n" + "=" * 80)
print("DATASET SUMMARY")
print("=" * 80)
info_df = pd.DataFrame(datasets_info)
print(info_df.to_string(index=False))
print(f"\nSaved: {EXPERIMENTS_DIR / 'eda_plots.png'}")

## 3. TF-IDF Baselines

We train Logistic Regression and Linear SVM models on each source dataset and evaluate on all target datasets.
Metrics: Accuracy, F1, ROC-AUC, PR-AUC.

In [ ]:
# @title Run TF-IDF Experiments
RUN_TFIDF = True # @param {type:"boolean"}

datasets = ["civil", "hatexplain", "jigsaw"]

if RUN_TFIDF:
    print("Running TF-IDF Baselines...")
    for source in datasets:
        # Target all other datasets + self
        targets = [d for d in datasets]
        
        print(f"\n--- Training on {source} ---")
        cmd = [
            sys.executable, "scripts/run_tfidf_baselines.py",
            "--source_dataset", source,
            "--target_datasets"
        ] + targets + [
            "--model", "both",  # Run both LogReg and SVM
            "--save_preds"
        ]
        
        print(f"Executing: {' '.join(cmd)}")
        subprocess.check_call(cmd)
        
    print("\nTF-IDF Experiments Completed.")
else:
    print("Skipping TF-IDF experiments.")

## 4. RoBERTa Experiments

We train RoBERTa-base models.
Options:
- **Standard**: Fine-tuning on source.
- **CORAL**: Domain adaptation (requires unlabeled target).
- **LoRA**: Parameter-efficient fine-tuning.

We also enable **Calibration** (Temperature Scaling) and save predictions for fairness analysis.

In [ ]:
# @title Run RoBERTa Experiments
RUN_ROBERTA = True # @param {type:"boolean"}
USE_CORAL = False # @param {type:"boolean"}
USE_LORA = False # @param {type:"boolean"}
FAST_MODE = True # @param {type:"boolean"}

# Configuration
EPOCHS = 1 if FAST_MODE else 3
SEEDS = [42] # Add more seeds for full paper results, e.g. [42, 123, 456]
BATCH_SIZE = 16
MAX_LEN = 128

if RUN_ROBERTA:
    print("Running RoBERTa Experiments...")
    
    for source in datasets:
        targets = [d for d in datasets] # All datasets
        
        # Base arguments
        cmd = [
            sys.executable, "scripts/run_roberta.py",
            "--source_dataset", source,
            "--model_name", "roberta-base",
            "--epochs", str(EPOCHS),
            "--batch_size", str(BATCH_SIZE),
            "--max_len", str(MAX_LEN),
            "--seeds"
        ] + [str(s) for s in SEEDS] + [
            "--target_datasets"
        ] + targets + [
            "--calibration", "temperature", # Enable calibration
            "--save_preds",
            "--amp", # Use Mixed Precision
            "--tune_threshold"
        ]
        
        # LoRA
        if USE_LORA:
            cmd += ["--peft", "lora"]
            
        # CORAL (Example: if source is civil, adapt to jigsaw)
        # Note: CORAL requires a specific target. For simplicity, we run standard here.
        # To run CORAL, you would need a separate loop or logic.
        
        print(f"\n--- Training RoBERTa on {source} ---")
        print(f"Executing: {' '.join(cmd)}")
        
        # Run
        subprocess.check_call(cmd)
        
    print("\nRoBERTa Experiments Completed.")
else:
    print("Skipping RoBERTa experiments.")

## 5. Fairness Analysis

We compute fairness metrics (Demographic Parity, Equal Opportunity) for models on datasets with identity attributes (Jigsaw, Civil).
We use the `scripts/fairness_metrics.py` script.

## 4.5 Calibration Analysis

Analyze model calibration using Expected Calibration Error (ECE) and reliability diagrams.

In [ ]:
# @title Calibration Analysis: ECE and Reliability Diagrams
from sklearn.calibration import calibration_curve

def compute_ece(y_true, y_prob, n_bins=10):
    """Compute Expected Calibration Error."""
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    ece = 0
    for i in range(n_bins):
        in_bin = (y_prob > bin_boundaries[i]) & (y_prob <= bin_boundaries[i + 1])
        prop_in_bin = in_bin.mean()
        if prop_in_bin > 0:
            avg_confidence = y_prob[in_bin].mean()
            avg_accuracy = y_true[in_bin].mean()
            ece += np.abs(avg_confidence - avg_accuracy) * prop_in_bin
    return ece

# Find prediction files
pred_files = list(EXPERIMENTS_DIR.glob("preds_*_test.csv"))

if pred_files:
    fig, axes = plt.subplots(1, len(pred_files), figsize=(5 * len(pred_files), 5))
    if len(pred_files) == 1:
        axes = [axes]
    
    calibration_results = []
    
    for ax, pred_file in zip(axes, pred_files):
        try:
            df = pd.read_csv(pred_file)
            if "pos_prob" not in df.columns and "score" not in df.columns:
                continue
            
            prob_col = "pos_prob" if "pos_prob" in df.columns else "score"
            y_true = df["label"].values
            y_prob = df[prob_col].values
            
            # Compute calibration curve
            prob_true, prob_pred = calibration_curve(y_true, y_prob, n_bins=10, strategy="uniform")
            
            # Compute ECE
            ece = compute_ece(y_true, y_prob)
            
            # Plot
            ax.plot([0, 1], [0, 1], "k--", label="Perfectly Calibrated")
            ax.plot(prob_pred, prob_true, "o-", label=f"Model (ECE={ece:.3f})")
            ax.fill_between(prob_pred, prob_pred, prob_true, alpha=0.2)
            ax.set_xlabel("Mean Predicted Probability")
            ax.set_ylabel("Fraction of Positives")
            ax.set_title(f"{pred_file.stem}")
            ax.legend(loc="lower right")
            ax.set_xlim([0, 1])
            ax.set_ylim([0, 1])
            
            calibration_results.append({
                "Model": pred_file.stem,
                "ECE": ece,
                "Brier": ((y_prob - y_true) ** 2).mean()
            })
        except Exception as e:
            print(f"Error processing {pred_file}: {e}")
    
    plt.tight_layout()
    plt.savefig(EXPERIMENTS_DIR / "calibration_reliability.png", dpi=150, bbox_inches="tight")
    plt.show()
    
    # Print calibration summary
    if calibration_results:
        cal_df = pd.DataFrame(calibration_results)
        print("\nCalibration Metrics:")
        print(cal_df.to_string(index=False))
        cal_df.to_csv(EXPERIMENTS_DIR / "calibration_metrics.csv", index=False)
        print(f"\nSaved: {EXPERIMENTS_DIR / 'calibration_reliability.png'}")
        print(f"Saved: {EXPERIMENTS_DIR / 'calibration_metrics.csv'}")
else:
    print("No prediction files found. Run experiments first.")

In [ ]:
# @title Run Fairness Metrics
RUN_FAIRNESS = True # @param {type:"boolean"}

# Define which datasets have identity labels and which split to use
# NOTE: Only Jigsaw has g_* identity columns in *_full.csv files
# Civil Comments full files only have 'toxicity' column, no identity groups
fairness_targets = [
    {"dataset": "jigsaw", "split": "test", "full_file": "jigsaw_test_full.csv", "group_prefix": "g_"},
]

if RUN_FAIRNESS:
    print("Running Fairness Analysis...")
    print("Note: Only Jigsaw has identity group columns for fairness analysis.\n")
    
    for target in fairness_targets:
        dataset = target["dataset"]
        split = target["split"]
        full_file = DATA_DIR / target["full_file"]
        
        if not full_file.exists():
            print(f"Skipping {dataset}: Full data file {full_file} not found.")
            continue
            
        # Check for predictions from RoBERTa (in-domain and cross-domain)
        # We look for preds_{source}_{dataset}.csv where dataset is the target
        # For in-domain: preds_{dataset}_test.csv
        
        # 1. In-domain RoBERTa
        pred_file = EXPERIMENTS_DIR / f"preds_{dataset}_test.csv"
        if pred_file.exists():
            print(f"\n--- Fairness: {dataset} (RoBERTa In-domain) ---")
            out_prefix = EXPERIMENTS_DIR / f"fairness_{dataset}_roberta_indomain"
            cmd = [
                sys.executable, "scripts/fairness_metrics.py",
                "--dataset", dataset,
                "--split", split,
                "--pred_file", str(pred_file),
                "--full_data_file", str(full_file),
                "--group_prefix", target["group_prefix"],
                "--out_prefix", str(out_prefix)
            ]
            subprocess.check_call(cmd)
        else:
            print(f"RoBERTa predictions not found for {dataset} in-domain: {pred_file}")

        # 2. In-domain TF-IDF (LogReg)
        pred_file = EXPERIMENTS_DIR / f"preds_tfidf_logreg_{dataset}_test.csv"
        if pred_file.exists():
            print(f"\n--- Fairness: {dataset} (TF-IDF LogReg In-domain) ---")
            out_prefix = EXPERIMENTS_DIR / f"fairness_{dataset}_tfidf_logreg_indomain"
            cmd = [
                sys.executable, "scripts/fairness_metrics.py",
                "--dataset", dataset,
                "--split", split,
                "--pred_file", str(pred_file),
                "--full_data_file", str(full_file),
                "--group_prefix", target["group_prefix"],
                "--out_prefix", str(out_prefix)
            ]
            subprocess.check_call(cmd)

        # 3. Cross-domain (e.g., Civil -> Jigsaw, HateXplain -> Jigsaw)
        for source in datasets:
            if source == dataset: 
                continue
            
            # RoBERTa cross-domain
            pred_file = EXPERIMENTS_DIR / f"preds_{source}_to_{dataset}.csv"
            if pred_file.exists():
                print(f"\n--- Fairness: {source} -> {dataset} (RoBERTa) ---")
                out_prefix = EXPERIMENTS_DIR / f"fairness_{source}_to_{dataset}_roberta"
                cmd = [
                    sys.executable, "scripts/fairness_metrics.py",
                    "--dataset", dataset,
                    "--split", split,
                    "--pred_file", str(pred_file),
                    "--full_data_file", str(full_file),
                    "--group_prefix", target["group_prefix"],
                    "--out_prefix", str(out_prefix)
                ]
                subprocess.check_call(cmd)
            
            # TF-IDF cross-domain
            pred_file = EXPERIMENTS_DIR / f"preds_tfidf_logreg_{source}_to_{dataset}.csv"
            if pred_file.exists():
                print(f"\n--- Fairness: {source} -> {dataset} (TF-IDF LogReg) ---")
                out_prefix = EXPERIMENTS_DIR / f"fairness_{source}_to_{dataset}_tfidf_logreg"
                cmd = [
                    sys.executable, "scripts/fairness_metrics.py",
                    "--dataset", dataset,
                    "--split", split,
                    "--pred_file", str(pred_file),
                    "--full_data_file", str(full_file),
                    "--group_prefix", target["group_prefix"],
                    "--out_prefix", str(out_prefix)
                ]
                subprocess.check_call(cmd)
            
    print("\nFairness Analysis Completed.")
else:
    print("Skipping Fairness Analysis.")

## 6. Results & Analysis

We aggregate results from all experiments and visualize:
1.  **Cross-Domain Performance**: F1 Score Heatmap.
2.  **Fairness Gaps**: Demographic Parity and Equal Opportunity differences.

In [ ]:
# @title Aggregate Results
import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load TF-IDF Summaries
tfidf_files = glob.glob(str(EXPERIMENTS_DIR / "summary_tfidf_*.csv"))
roberta_files = glob.glob(str(EXPERIMENTS_DIR / "summary_*.csv"))

all_metrics = []

# Process TF-IDF
for f in tfidf_files:
    try:
        df = pd.read_csv(f)
        # Filename: summary_tfidf_{source}_{model}.csv
        fname = Path(f).stem
        parts = fname.split("_")
        # parts: ['summary', 'tfidf', source, model]
        if len(parts) >= 4:
            source = parts[2]
            model = parts[3]
        else:
            continue
            
        for _, row in df.iterrows():
            split = row["split"]
            if split == "in_domain_test":
                target = source
            elif split.startswith("cross_"):
                target = split.replace("cross_", "")
            else:
                continue
                
            all_metrics.append({
                "Model": f"TF-IDF ({model})",
                "Source": source,
                "Target": target,
                "F1": row.get("f1", 0),
                "Accuracy": row.get("accuracy", 0),
                "AUROC": row.get("auroc", 0)
            })
    except Exception as e:
        print(f"Error reading {f}: {e}")

# Process RoBERTa
for f in roberta_files:
    if "tfidf" in f: continue # Skip tfidf here
    try:
        df = pd.read_csv(f)
        source = Path(f).stem.replace("summary_", "")
        
        for _, row in df.iterrows():
            split = row["split"]
            if split == "in_domain_test":
                target = source
            elif split.startswith("cross_"):
                target = split.replace("cross_", "")
            else:
                continue
                
            all_metrics.append({
                "Model": "RoBERTa",
                "Source": source,
                "Target": target,
                "F1": row.get("f1", 0),
                "Accuracy": row.get("accuracy", 0),
                "AUROC": row.get("auroc", 0)
            })
    except Exception as e:
        print(f"Error reading {f}: {e}")

results_df = pd.DataFrame(all_metrics)

if not results_df.empty:
    print("Loaded Results:")
    print(results_df.head())
else:
    print("No results found.")

In [ ]:
# @title Plot Cross-Domain Performance Heatmaps
if not results_df.empty:
    models = results_df["Model"].unique()
    n_models = len(models)
    
    fig, axes = plt.subplots(1, n_models, figsize=(6 * n_models, 5))
    if n_models == 1:
        axes = [axes]
    
    for ax, model in zip(axes, models):
        model_data = results_df[results_df["Model"] == model]
        if model_data.empty:
            continue
        pivot_table = model_data.pivot(index="Source", columns="Target", values="F1")
        sns.heatmap(pivot_table, annot=True, cmap="viridis", fmt=".3f", ax=ax, vmin=0, vmax=1)
        ax.set_title(f"{model} Cross-Domain F1 Score")
    
    plt.tight_layout()
    plt.savefig(EXPERIMENTS_DIR / "cross_domain_f1_heatmap.png", dpi=150, bbox_inches="tight")
    plt.show()
    print(f"Saved: {EXPERIMENTS_DIR / 'cross_domain_f1_heatmap.png'}")
else:
    print("No results to plot.")

In [ ]:
# @title Plot ROC Curves Comparison
from sklearn.metrics import roc_curve, auc

# Find all prediction files for ROC curves
all_pred_files = list(EXPERIMENTS_DIR.glob("preds_*.csv"))

if all_pred_files:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Group by target dataset
    for idx, target_dataset in enumerate(["civil", "hatexplain", "jigsaw"]):
        ax = axes[idx]
        ax.plot([0, 1], [0, 1], 'k--', lw=2, label='Random')
        
        # Find all predictions that evaluate on this target
        for pred_file in all_pred_files:
            fname = pred_file.stem
            
            # Check if this file evaluates on target_dataset
            is_match = False
            if f"_to_{target_dataset}" in fname:
                is_match = True
                label = fname.replace("preds_", "").replace(f"_to_{target_dataset}", f"→{target_dataset}")
            elif fname == f"preds_{target_dataset}_test":
                is_match = True
                label = f"{target_dataset} (in-domain)"
            elif fname == f"preds_tfidf_logreg_{target_dataset}_test":
                is_match = True
                label = f"TF-IDF {target_dataset} (in-domain)"
            
            if not is_match:
                continue
            
            try:
                df = pd.read_csv(pred_file)
                prob_col = "pos_prob" if "pos_prob" in df.columns else "score"
                if prob_col not in df.columns:
                    continue
                
                y_true = df["label"].values
                y_score = df[prob_col].values
                
                fpr, tpr, _ = roc_curve(y_true, y_score)
                roc_auc = auc(fpr, tpr)
                
                ax.plot(fpr, tpr, lw=2, label=f'{label} (AUC={roc_auc:.3f})')
            except Exception as e:
                print(f"Error with {pred_file}: {e}")
        
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title(f'ROC Curves: Target = {target_dataset.upper()}')
        ax.legend(loc='lower right', fontsize=8)
    
    plt.tight_layout()
    plt.savefig(EXPERIMENTS_DIR / "roc_curves.png", dpi=150, bbox_inches="tight")
    plt.show()
    print(f"Saved: {EXPERIMENTS_DIR / 'roc_curves.png'}")
else:
    print("No prediction files found for ROC curves.")

In [ ]:
# @title Plot Fairness Gaps
fairness_files = glob.glob(str(EXPERIMENTS_DIR / "fairness_*_summary.csv"))

fairness_data = []

for f in fairness_files:
    try:
        df = pd.read_csv(f)
        # Filename: fairness_{context}_summary.csv
        fname = Path(f).stem
        context = fname.replace("fairness_", "").replace("_summary", "")
        
        # Get max gaps across all identity groups
        max_dp = df["dp_diff"].max()
        max_eop = df["eop_diff"].max()
        max_eo = df["eo_diff"].max()
        
        fairness_data.append({
            "Setting": context,
            "Max DP Gap": max_dp,
            "Max EOp Gap": max_eop,
            "Max EO Gap": max_eo
        })
    except Exception as e:
        print(f"Error reading {f}: {e}")

if fairness_data:
    f_df = pd.DataFrame(fairness_data)
    print("\nFairness Gaps Summary:")
    print(f_df.to_string(index=False))
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 6))
    f_df.set_index("Setting")[["Max DP Gap", "Max EOp Gap", "Max EO Gap"]].plot(kind="bar", ax=ax)
    ax.set_title("Fairness Gaps by Setting (Lower is Better)")
    ax.set_ylabel("Gap")
    ax.set_xlabel("Model / Domain Setting")
    plt.xticks(rotation=45, ha="right")
    plt.legend(title="Metric")
    plt.tight_layout()
    plt.savefig(EXPERIMENTS_DIR / "fairness_gaps.png", dpi=150, bbox_inches="tight")
    plt.show()
    print(f"Saved: {EXPERIMENTS_DIR / 'fairness_gaps.png'}")
else:
    print("No fairness results found.")

## 7. Summary Tables for Report

Generate formatted tables ready for the final report.

In [ ]:
# @title Generate Summary Tables
if not results_df.empty:
    # Table 1: In-Domain Performance
    print("=" * 80)
    print("TABLE 1: In-Domain Performance (Source = Target)")
    print("=" * 80)
    in_domain = results_df[results_df["Source"] == results_df["Target"]]
    in_domain_pivot = in_domain.pivot(index="Source", columns="Model", values=["F1", "Accuracy", "AUROC"])
    print(in_domain_pivot.round(4).to_string())
    
    # Table 2: Cross-Domain Performance (F1)
    print("\n" + "=" * 80)
    print("TABLE 2: Cross-Domain F1 Score")
    print("=" * 80)
    for model in results_df["Model"].unique():
        print(f"\n{model}:")
        model_data = results_df[results_df["Model"] == model]
        pivot = model_data.pivot(index="Source", columns="Target", values="F1")
        print(pivot.round(4).to_string())
    
    # Table 3: OOD Performance Drop
    print("\n" + "=" * 80)
    print("TABLE 3: OOD Performance Drop (In-Domain F1 - Cross-Domain F1)")
    print("=" * 80)
    for model in results_df["Model"].unique():
        print(f"\n{model}:")
        model_data = results_df[results_df["Model"] == model]
        pivot = model_data.pivot(index="Source", columns="Target", values="F1")
        # Calculate drop from diagonal (in-domain)
        for src in pivot.index:
            if src in pivot.columns:
                in_domain_f1 = pivot.loc[src, src]
                for tgt in pivot.columns:
                    if src != tgt:
                        pivot.loc[src, tgt] = in_domain_f1 - pivot.loc[src, tgt]
                pivot.loc[src, src] = 0  # No drop for in-domain
        print(pivot.round(4).to_string())
    
    # Save to CSV
    results_df.to_csv(EXPERIMENTS_DIR / "all_results_combined.csv", index=False)
    print(f"\nSaved all results to: {EXPERIMENTS_DIR / 'all_results_combined.csv'}")
else:
    print("No results available.")

In [ ]:
# @title Export Results for Report (LaTeX Tables)
def df_to_latex(df, caption="", label=""):
    """Convert DataFrame to LaTeX table format."""
    latex = df.to_latex(float_format="%.4f", escape=False)
    if caption:
        latex = latex.replace("\\begin{tabular}", f"\\caption{{{caption}}}\n\\label{{{label}}}\n\\begin{tabular}")
    return latex

if not results_df.empty:
    # Export main results table
    for model in results_df["Model"].unique():
        model_data = results_df[results_df["Model"] == model]
        pivot = model_data.pivot(index="Source", columns="Target", values="F1")
        
        model_name_clean = model.replace(" ", "_").replace("(", "").replace(")", "")
        latex_file = EXPERIMENTS_DIR / f"table_{model_name_clean}_f1.tex"
        
        with open(latex_file, "w") as f:
            f.write(df_to_latex(pivot, 
                               caption=f"{model} Cross-Domain F1 Scores",
                               label=f"tab:{model_name_clean}_f1"))
        print(f"Saved LaTeX table: {latex_file}")

# Export fairness table
if fairness_data:
    f_df = pd.DataFrame(fairness_data)
    latex_file = EXPERIMENTS_DIR / "table_fairness.tex"
    with open(latex_file, "w") as f:
        f.write(df_to_latex(f_df.set_index("Setting"), 
                           caption="Fairness Gaps Across Models and Domains",
                           label="tab:fairness"))
    print(f"Saved LaTeX table: {latex_file}")

print("\n✓ All exports complete!")

## 8. Output Verification & Checklist

Verify all expected outputs were generated.

In [ ]:
# @title Verify All Outputs
print("=" * 80)
print("OUTPUT VERIFICATION CHECKLIST")
print("=" * 80)

expected_outputs = {
    "Plots": [
        "eda_plots.png",
        "cross_domain_f1_heatmap.png",
        "roc_curves.png",
        "calibration_reliability.png",
        "fairness_gaps.png",
    ],
    "Tables (CSV)": [
        "all_results_combined.csv",
        "calibration_metrics.csv",
    ],
    "LaTeX Tables": [
        "table_RoBERTa_f1.tex",
        "table_TF-IDF_logreg_f1.tex",
        "table_TF-IDF_svm_f1.tex",
        "table_fairness.tex",
    ],
    "TF-IDF Summaries": [
        "summary_tfidf_civil_logreg.csv",
        "summary_tfidf_civil_svm.csv",
        "summary_tfidf_hatexplain_logreg.csv",
        "summary_tfidf_hatexplain_svm.csv",
        "summary_tfidf_jigsaw_logreg.csv",
        "summary_tfidf_jigsaw_svm.csv",
    ],
    "RoBERTa Summaries": [
        "summary_civil.csv",
        "summary_hatexplain.csv",
        "summary_jigsaw.csv",
    ],
    "Fairness Results": [
        "fairness_jigsaw_roberta_indomain_summary.csv",
        "fairness_jigsaw_tfidf_logreg_indomain_summary.csv",
    ],
}

total_expected = 0
total_found = 0

for category, files in expected_outputs.items():
    print(f"\n{category}:")
    for f in files:
        total_expected += 1
        path = EXPERIMENTS_DIR / f
        if path.exists():
            print(f"  ✓ {f}")
            total_found += 1
        else:
            print(f"  ✗ {f} (MISSING)")

print(f"\n{'=' * 80}")
print(f"SUMMARY: {total_found}/{total_expected} files generated")
if total_found == total_expected:
    print("✓ All outputs verified!")
else:
    print(f"⚠️  {total_expected - total_found} files missing. Re-run relevant cells.")
print("=" * 80)

## 9. Usage Notes

### Running on Google Colab Pro
1. Upload the entire repository to Colab or mount from Google Drive
2. Ensure GPU runtime is enabled: `Runtime → Change runtime type → GPU`
3. Run cells in order, adjusting toggles:
   - `RUN_TFIDF = True` → Runs TF-IDF baselines (~5-10 min)
   - `RUN_ROBERTA = True` → Runs RoBERTa experiments (~30-60 min per dataset with GPU)
   - `FAST_MODE = True` → Uses 1 epoch for quick testing
   - `RUN_FAIRNESS = True` → Computes fairness metrics on Jigsaw dataset

### Expected Outputs for Report
| Output Type | Files | Description |
|-------------|-------|-------------|
| **EDA** | `eda_plots.png` | Dataset statistics, label distributions |
| **Performance** | `cross_domain_f1_heatmap.png` | Cross-domain F1 score matrix |
| **ROC Curves** | `roc_curves.png` | Model comparison by AUC |
| **Calibration** | `calibration_reliability.png`, `calibration_metrics.csv` | ECE, Brier scores |
| **Fairness** | `fairness_gaps.png`, `fairness_*_summary.csv` | DP, EOp, EO gaps |
| **Tables** | `table_*.tex` | LaTeX-ready tables |
| **Raw Data** | `all_results_combined.csv` | All metrics in one file |

### Key Findings to Report
1. **In-Domain Performance**: How well models perform on their training distribution
2. **Cross-Domain Generalization**: Performance drop when tested on different datasets  
3. **Calibration**: Model confidence vs actual accuracy (ECE, Brier)
4. **Fairness**: Demographic parity and equal opportunity gaps across identity groups